# **Linear Regression**

**Goal**: leverage 18 features (including PM2.5) in the first 9 hours to predict PM2.5 in the 10th hour.

# **Load `train.csv`**
- The `train.csv` file contains data from 12 months, 20 days of for each month, and 24 hours a day (hourly data with 18 features).

In [ ]:
import sys
import pandas as pd
import numpy as np
from google.colab import drive 
!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
!unzip data.zip
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
data = pd.read_csv('./train.csv', encoding = 'big5')

# **Preprocessing** 
- Extract features.
- Replace data in `RAINFALL` column with `0`.

In [ ]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

# **Extract Features (1)**
![圖片說明](https://drive.google.com/uc?id=1LyaqD4ojX07oe5oDzPO99l9ts5NRyArH)
![圖片說明](https://drive.google.com/uc?id=1ZroBarcnlsr85gibeqEF-MtY13xJTG47)

- Regroup the original 4,320 data points (with 18-features) into 12 windows. Each window is one month containing data of 18 features and 480 hours.

In [ ]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

# **Extract Features (2)**
![alt text](https://drive.google.com/uc?id=1wKoPuaRHoX682LMiBgIoOP4PDyNKsJLK)
![alt text](https://drive.google.com/uc?id=1FRWWiXQ-Qh0i9tyx0LiugHYF_xDdkhLN)

- There will be 480hrs-data (24 \* 10) every month, and **a data** is formed every 9 hours. Also, there will be 471 data every month, so the total number of data is 471 \* 12. Each data has 9 (hours) \* 18 (features).
- The corresponding target size is 471 \* 12 (PM2.5 in the 10th hour).

In [ ]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

# **Normalize (1)**


In [ ]:
mean_x = np.mean(x, axis = 0) # 18 * 9 
std_x = np.std(x, axis = 0) # 18 * 9 
for i in range(len(x)): # 12 * 471
    for j in range(len(x[0])): # 18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

# **Split Training Data Into `train_set` and `validation_set`**

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

# **Training**
![alt text](https://drive.google.com/uc?id=1xIXvqZ4EGgmxrp7c9r0LOVbcvd4d9H4N)
![alt text](https://drive.google.com/uc?id=1S42g06ON5oJlV2f9RukxawjbE4NpsaB6)
![alt text](https://drive.google.com/uc?id=1BbXu-oPB9EZBHDQ12YCkYqtyAIil3bGj)

- Different from the above picture: the code below uses *Root Mean Square Error*.

- Because of the existence of the constant term, dimension (`dim`) needs to be added one more column; the `eps` term is a minimal value added to avoid the denominator of adagrad being `0`.

- Each dimension (`dim`) will correspond to its own gradient, weight (`w`), and learn through iteration (`iter_time`).

In [ ]:
dim = 18 * 9 + 1 # The dimension of all parameters, `+1` is to handle the bias
w = np.zeros([dim, 1])
# `np.ones` generates an array of 12*471 columns and 1 row and merges with the original features
# That is, add a new vector to the first row to become an array with 12*471 columns and 18*9+1
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)

learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1]) # an array to update the learning rate for adagrad
eps = 0.0000000001 # Avoid the denominator of iterative learning rate lr/sqrt(sum_of_pre_grads**2) becomes 0
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12) # rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) # dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

# **Testing**
![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

- Load the test set and preprocess it in a manner similar to the training data, so that the test data forms 240 data with dimensions of 18 \* 9 + 1.

In [ ]:
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

# **Prediction**

![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

- With weight and test set, the target can be predicted.

In [ ]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

# **Save Prediction to CSV File**


In [ ]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

Related reference:

- [Adagrad](https://youtu.be/yKKNr-QKz2Q?list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&t=705)

- [RMSprop](https://www.youtube.com/watch?v=5Yt-obwvMHI)

- [Adam](https://www.youtube.com/watch?v=JXQT_vxqwIs)


The `print` above is mainly to see the results, it doesn't hurt to remove it.

Finally, you can surpass the baseline by adjusting the learning rate, iter_time (number of iterations), the number of features used (how many hours are taken, which feature fields to take), and even different models.